In [1]:
#   SDK
import relevanceai

project = "michelangioloma" #'638c33786033a2ebd49f'
api_key = "TDFvdm9Yd0J1M3VVTEcyQTV3VWs6b3RnRXJnUW9SQ2kwMVVJWVZ2VzVtZw" #'TGx6UGNIWUJ5R0lBMktUNTRsbjM6VElZRzhaRjlRUnVYY3RfQ0pISXd1QQ'
client = relevanceai.Client(project, api_key)
client.base_url = "https://gateway-api-aueast.relevance.ai/latest"

C:\Users\ardit\AppData\Local\Programs\Python\Python39\lib\site-packages\relevanceai\__init__.py:23: UserWarning: Your RelevanceAI version (0.25.0) is not the latest. Please install the latest version (0.26.6) by running pip install -U relevanceai
  warnings.warn(


In [2]:
import json

#import json
with open('CRL_4_df_ready.json') as file:
    df_ready = json.loads(file.read())
df_ready

[{'_id': 0,
  'deck_vector_': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  'deck_original': ['Ram Rider',
   'Inferno Dragon',
   'Lumberjack',
   'Fireball',
   'Bandit',
   'Magic Archer',
   'B

In [3]:
#upload df
client.datasets.delete(dataset_id='crl_matches')
client.datasets.bulk_insert(dataset_id='crl_matches', documents=df_ready)

{'inserted': 23640, 'failed_documents': []}

In [4]:
client.datasets.schema('crl_matches')

{'crown_opponent': 'numeric',
 'crown_team': 'numeric',
 'deck_original': 'text',
 'deck_vector_': {'vector': 106},
 'insert_date_': 'date',
 'links': 'text',
 'result': 'text'}

## clustering

In [5]:
# Vector field based on which clustering is done - (Currently only one vector is supported)
vector_field = 'descriptiontextmulti_vector_'

#local or remote?
centroids = client.vector_tools.cluster.kmeans_cluster(
    dataset_id = 'crl_matches', 
    vector_fields=['deck_vector_'],
    k = 12)

#creates clusters but only gives the centroids
#clustering results is uploaded on the database

100%|██████████| 24/24 [00:56<00:00,  2.34s/it]


Initialization complete
Iteration 0, inertia 185432.0
Iteration 1, inertia 114477.51335372643
Iteration 2, inertia 112188.11912804336
Iteration 3, inertia 110838.90174653212
Iteration 4, inertia 110501.16323655371
Iteration 5, inertia 110359.18229692428
Iteration 6, inertia 110339.02059981576
Iteration 7, inertia 110338.69237548117
Iteration 8, inertia 110338.22005620832
Iteration 9, inertia 110337.99315374104
Converged at iteration 9: strict convergence.
Initialization complete
Iteration 0, inertia 181970.0
Iteration 1, inertia 115033.88950357665
Iteration 2, inertia 113082.95708289019
Iteration 3, inertia 112418.13574806886
Iteration 4, inertia 112126.86610945861
Iteration 5, inertia 111994.05084922544
Iteration 6, inertia 111918.24453648353
Iteration 7, inertia 111892.26874621489
Iteration 8, inertia 111885.50353978643
Iteration 9, inertia 111884.4976080973
Iteration 10, inertia 111884.46577502078
Iteration 11, inertia 111884.43293211785
Converged at iteration 11: strict convergence

In [6]:
client.datasets.schema('crl_matches')

{'_cluster_': 'dict',
 '_cluster_.deck_vector_': 'dict',
 '_cluster_.deck_vector_.kmeans_12': 'text',
 'crown_opponent': 'numeric',
 'crown_team': 'numeric',
 'deck_original': 'text',
 'deck_vector_': {'vector': 106},
 'insert_date_': 'date',
 'links': 'text',
 'result': 'text'}

## dashboard

In [8]:
client.services.cluster.centroids.list_closest_to_center(
  dataset_id='crl_matches',
  vector_fields=["deck_vector_"],
  page_size=20,
  #cluster_ids=[], # Leave this as an empty list if you want all of the clusters
  alias="kmeans_12" #change to 'kmeans_10' 
)

You can now visit the dashboard at https://cloud.relevance.ai/sdk/cluster/centroids/closest


{'Cluster-0': [{'_id': '18017',
   'result': 'lose',
   'crown_opponent': 3,
   'deck_original': ['Royal Giant',
    'Mother Witch',
    'Mega Minion',
    'Fireball',
    'Fisherman',
    'Goblin Cage',
    'Electro Spirit',
    'The Log'],
   'insert_date_': '2022-01-10T04:26:11.595Z',
   '_cluster_': {},
   'links': ['https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/royal_giant.png',
    'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/motherwitch.png',
    'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/mega_minion.png',
    'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/fire_fireball.png',
    'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_challenge/CRL_images/fisherman.png',
    'https://raw.githubusercontent.com/arditoibryan/Projects/master/20210113_CRL_c